In [16]:
import os
import io
import IPython.display
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HF_API_KEY']

In [11]:
import requests, json

API_URL = "https://api-inference.huggingface.co/models/Salesforce/blip-image-captioning-base"
headers = {"Authorization": hf_api_key}


In [13]:
def query(filename):
    with open(filename, "rb") as f:
        data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    return response.json()

output = query("./alien.jpg")

In [10]:
def get_completion(inputs, parameters=None, ENDPOINT_URL=API_URL):
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL,
                                headers=headers,
                                data=json.dumps(data))
    return json.loads(response.content.decode("utf-8"))

In [23]:
image_url = "https://www.livemint.com/lm-img/img/2023/10/01/600x338/2-0-94332521-000-Del173249-0_1681792822720_1696170349986.jpg"
# https://free-images.com/lg/9cb8/sunset_sundown_da_nang.jpg
# https://free-images.com/lg/dd2c/port_au_prince_haiti.jpg
# https://free-images.com/lg/08a8/jacques_louis_david_coronation_1.jpg
display(IPython.display.Image(url=image_url))
get_completion(image_url)

[{'generated_text': 'a group of people standing in front of a tree'}]

In [28]:
! pip install gradio

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instagrapi 1.17.1 requires pydantic==1.10.5, but you have pydantic 2.6.2 which is incompatible.
langchain 0.0.208 requires pydantic<2,>=1, but you have pydantic 2.6.2 which is incompatible.
langchainplus-sdk 0.0.20 requires pydantic<2,>=1, but you have pydantic 2.6.2 which is incompatible.
python-telegram-bot 20.2 requires httpx~=0.23.3, but you have httpx 0.27.0 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\


  Using cached gradio-4.19.2-py3-none-any.whl.metadata (15 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached fastapi-0.110.0-py3-none-any.whl.metadata (25 kB)
  Using cached ffmpy-0.3.2-py3-none-any.whl
  Using cached gradio_client-0.10.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached importlib_resources-6.1.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached orjson-3.9.15-cp310-none-win_amd64.whl.metadata (50 kB)
  Using cached pydantic-2.6.2-py3-none-any.whl.metadata (83 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached ruff-0.2.2-py3-none-win_amd64.whl.metadata (23 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached tomlkit-0.12.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached uvicorn-0.27.1-py3-none-any.whl.metadata (6.3 kB)
  Using cach

In [35]:
import streamlit as st
import gradio as gr

In [37]:

def image_to_base64_str(pil_image):
    byte_arr = io.BytesIO()
    pil_image.save(byte_arr, format='PNG')
    byte_arr = byte_arr.getvalue()
    return str(base64.b64encode(byte_arr).decode('utf-8'))

def captioner(image):
    base64_image = image_to_base64_str(image)
    result = get_completion(base64_image)
    return result[0]['generated_text']

gr.close_all()
demo = gr.Interface(fn=captioner,
                    inputs=[gr.Image(label="Upload image", type="pil")],
                    outputs=[gr.Textbox(label="Caption")],
                    title="Image Captioning with BLIP",
                    description="Caption any image using the BLIP model",
                    allow_flagging="never",
                    examples=["https://free-images.com/lg/9cb8/sunset_sundown_da_nang.jpg", "https://free-images.com/lg/dd2c/port_au_prince_haiti.jpg"])

# demo.launch(share=True, server_port=int(os.environ['PORT1']))
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://4e05a97dda16bac375.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
